#  前端上传图片

前端上传图片大致分为两个步骤：

> 将上传的图片保存到服务器中——使用open的二进制读写方法。
> 
> 将存储在服务器上的图片路径保存到数据库中  

## 定义models类

定义models类，连接数据库，用于存储图片的路径，以便通过数据库存储的路径调取图片信息。

```python
class PicTest(models.Model):
    pic = models.ImageField(upload_to='demo/')
    # 数据库中存储图片后台路径
```

图片等均存储在多媒体目录中，在settings.py中设置多媒体连接路径  

```python
STATICFILES_DIRS=[
    os.path.join(BASE_DIR,'static')
]    # 静态文件连接路径

MEDIA_ROOT=os.path.join(BASE_DIR,'static/media')    # 多媒体文件链接路径
```



## 配置子路由

简单定义一个上传图片页面和一个展示图片页面，并实现相应的逻辑

```python
urlpatterns = [
    url(r'^$', index),
    url(r'^pic_handle/$', pic_handle),        # 上传图片路由
    url(r'^pic_show/$', pic_show),          # 展示图片路由
]
```

## 定义上传图片HTML页面

上传图片使用form表单进行上传，表单中必须指定`enctype="multipart/form-data"`多媒体属性。

```html
<form action="/pic_handle/" method="post" enctype="multipart/form-data">
    {% csrf_token %}
    <input type="file" name="pic">
    <br>
    <input type="submit" value="提交">
</form>
```

> 必须指定多媒体属性`enctype="multipart/form-data"`在form标签中
> 
> 使用`file`文件控件来上传图片
> 

## 定义上传图片的views函数

```python
from django.shortcuts import render, redirect
from .models import *
import os  # 导入os模块用于拼接路径
from django.conf import settings  # 导入settings用于获取多媒体目录

def pic_handle(request):
    pic = request.FILES.get('pic')
    # 获取前端file控件返回过来的值
    path = os.path.join(settings.MEDIA_ROOT, 'demo', pic.name)
    # 组装图片路径，以将他存储到后台指定位置(要把图片存储到static/meida/demo下)。
    # pic.name获取图片名

    # 使用with open方法将文件以二进制方式存储到指定位置(path提供指定位置)
    with open(path, 'bw') as f:
        for data in pic.chunks():
            # .chunks方法将文件转换成二进制数据流
            f.write(data)
            # 文件写入
    pic_url = 'booktest/{}'.format(pic.name)
    PicTest.objects.create(pic=pic_url)
    # 将图片后台路径存储到数据库中
        
    return redirect('/pic_show/')
```

> request.FILES.get()来获取前端file控件返回的值
> 
> path路径要拼接正确，存储到指定多媒体目录中
> 
> .chunks()将文件转换成二进制数据流格式
> 
> open使用二进制写方式
> 

## 展示图片HTML页面

```html
{% for foo in all_pic %}
    <img src="/static/media/{{ foo.pic }}" alt="">
    <br>
{% endfor %}

```

> 图片路径要确定清楚


## 定义展示图片的views函数

```python
def pic_show(request):
    all_pic = PicTest.objects.all()
    return render(request, 'demo/pic_show.html', locals())

```